In [5]:
import pandas as pd
import numpy as np
import math

In [6]:
df = pd.read_csv("card transactions.csv")
df = df[df.columns[:10]]

In [7]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,1/1/10,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,1/1/10,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,1/1/10,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96753 entries, 0 to 96752
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Recnum             96753 non-null  int64  
 1   Cardnum            96753 non-null  int64  
 2   Date               96753 non-null  object 
 3   Merchnum           93378 non-null  object 
 4   Merch description  96753 non-null  object 
 5   Merch state        95558 non-null  object 
 6   Merch zip          92097 non-null  float64
 7   Transtype          96753 non-null  object 
 8   Amount             96753 non-null  float64
 9   Fraud              96753 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 7.4+ MB


In [10]:
df_clean = df.copy()
df_clean.sort_values(by='Amount', ascending = False)#[1:].sort_values(by='Recnum')
#df_clean

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
52714,52715,5142189135,7/13/10,NaN,INTERMEXICO,NaN,NaN,P,3102045.53,0
47339,47340,5142275225,6/22/10,NaN,INTERSOL,NaN,NaN,P,47900.00,0
59516,59517,5142138135,8/3/10,900009045549,KRACKELER SCIENTIFIC,NY,12202.0,P,30372.46,1
80886,80887,5142152857,10/11/10,NaN,CANYON ELECTRIC,NV,89102.0,P,28392.84,1
89673,89674,5142183210,11/29/10,NaN,CRUCIAL TECH./MICRON,ID,83642.0,P,27218.00,0
...,...,...,...,...,...,...,...,...,...,...
47165,47166,5142148452,6/22/10,930009906224,FEDEX SHP 05/23/10 AB#,TN,38118.0,P,0.08,0
57855,57856,5142129787,7/30/10,9108347680000,STAPLES NATIONAL #471,NJ,7606.0,P,0.02,0
78387,78388,5142236687,9/28/10,9108233080006,STAPLES #990,CA,90640.0,P,0.01,0
91179,91180,5142239369,12/6/10,89200648125,JOHN WILEY & SON SOMERSET,NJ,8875.0,P,0.01,0


In [5]:
#Identify any exclusions, bad records

#1.remove single large transaction outlier
df_clean = df.copy()
df_clean = df_clean.sort_values(by='Amount', ascending = False)[1:].sort_values(by='Recnum')

#2. remove all but the “P” type transactions
df_clean = df_clean[df_clean.Transtype == 'P']

In [6]:
df_clean = df_clean.reset_index()
df_clean = df_clean[df_clean.columns[1:]]
df_clean.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,1/1/10,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,1/1/10,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,1/1/10,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [7]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96397 entries, 0 to 96396
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Recnum             96397 non-null  int64  
 1   Cardnum            96397 non-null  int64  
 2   Date               96397 non-null  object 
 3   Merchnum           93199 non-null  object 
 4   Merch description  96397 non-null  object 
 5   Merch state        95377 non-null  object 
 6   Merch zip          92097 non-null  float64
 7   Transtype          96397 non-null  object 
 8   Amount             96397 non-null  float64
 9   Fraud              96397 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 7.4+ MB


In [8]:
#fill in missing: Merch state
missing_state = np.where(pd.isnull(df_clean['Merch state']))[0]
print(len(missing_state))
#If the record has a zip, use the state for that zip, if known
zip_state = df_clean.groupby(['Merch zip', 'Merch state']).count().reset_index()[['Merch zip', 'Merch state']]
zip_state.drop_duplicates(subset ="Merch zip",keep = False, inplace = True)
zip_state = zip_state.reset_index()

miss_state_zip = []
for idx in missing_state:
    if math.isnan(df_clean.loc[idx, 'Merch zip']):
        miss_state_zip.append(idx)
    else:
        if df_clean.loc[idx, 'Merch zip'] in zip_state['Merch zip'].values:
            zipcode = df_clean.loc[idx, 'Merch zip']
            df_clean.loc[idx, 'Merch state'] = zip_state[zip_state['Merch zip'] == zipcode]['Merch state'].values[0]
            
        #If in range 00600 – 00799, 00900 – 00999: state = PR (Puerto Rico) 
        elif (df_clean.loc[idx, 'Merch zip'] >= 600 and df_clean.loc[idx, 'Merch zip'] <= 799) | (df_clean.loc[idx, 'Merch zip'] >= 900 and df_clean.loc[idx, 'Merch zip'] <= 999):
            df_clean.loc[idx, 'Merch state'] = 'PR'
        else:
            miss_state_zip.append(idx)
            
print(len(miss_state_zip))        


#Use the mode of the Merchnum or Merch description
merchnum_group = df_clean.groupby('Merchnum')['Merch state'].agg(pd.Series.mode).reset_index()
merdesc_group = df_clean.groupby('Merch description')['Merch state'].agg(pd.Series.mode).reset_index()

miss_merch = []
for idx in miss_state_zip:
    if df_clean.loc[idx, 'Merchnum'] in [np.nan] :
        if df_clean.loc[idx, 'Merch description'] in merdesc_group['Merch description'].values:
            descode = df_clean.loc[idx, 'Merch description']
            if type(merdesc_group[merdesc_group['Merch description'] == descode]['Merch state'].values[0]) == str:
                df_clean.loc[idx, 'Merch state'] = merdesc_group[merdesc_group['Merch description'] == descode]['Merch state'].values[0]
            else:
                miss_merch.append(idx)
        else:
            miss_merch.append(idx)
    else:
        if df_clean.loc[idx, 'Merchnum'] in merchnum_group['Merchnum'].values:
            numcode = df_clean.loc[idx, 'Merchnum']
            if type(merchnum_group[merchnum_group['Merchnum'] == numcode]['Merch state'].values[0]) == str:
                df_clean.loc[idx, 'Merch state'] = merchnum_group[merchnum_group['Merchnum'] == numcode]['Merch state'].values[0]
            else:
                miss_merch.append(idx)
                
        else:
            if df_clean.loc[idx, 'Merch description'] in merdesc_group['Merch description'].values:
                descode = df_clean.loc[idx, 'Merch description']
                if type(merdesc_group[merdesc_group['Merch description'] == descode]['Merch state'].values[0]) == str:
                    df_clean.loc[idx, 'Merch state'] = merdesc_group[merdesc_group['Merch description'] == descode]['Merch state'].values[0]
                else:
                    miss_merch.append(idx)
            else:
                miss_merch.append(idx)
                    
print(len(miss_merch)) 

#Fill in with “Unk” for unknown
for idx in miss_merch:
    df_clean.loc[idx, 'Merch state'] = 'Unk'

1020
977
293


In [9]:
sum(df_clean['Merch state'].isnull())

0

In [10]:
#fill in Merchnum:
#If = 0, replace with NaN
zero_idx = df_clean[(df_clean.Merchnum == '0')].index
print(len(zero_idx))
df_clean.loc[zero_idx, 'Merchnum'] = np.nan

#Fill in with mode of Merch description
missing_num = np.where(pd.isnull(df_clean['Merchnum']))[0]
print(len(missing_num))

merdesc_num = df_clean.groupby('Merch description')['Merchnum'].agg(pd.Series.mode).reset_index()

miss_desc = []
for idx in missing_num:
    if df_clean.loc[idx, 'Merch description'] in merdesc_num['Merch description'].values:
            descode = df_clean.loc[idx, 'Merch description']
            if type(merdesc_num[merdesc_num['Merch description'] == descode]['Merchnum'].values[0]) == str:
                df_clean.loc[idx, 'Merchnum'] = merdesc_num[merdesc_num['Merch description'] == descode]['Merchnum'].values[0]
            else:
                miss_desc.append(idx)
    else:
        miss_desc.append(idx)
        
print(len(miss_desc))

#Fill in with unknown
for idx in miss_desc:
    df_clean.loc[idx, 'Merchnum'] = 'Unk'

53
3251
2096


In [11]:
sum(df_clean['Merchnum'].isnull())

0

In [12]:
#fill in Merch zip
missing_zip = np.where(pd.isnull(df_clean['Merch zip']))[0]
print(len(missing_num))
#Fill in with mode of merchant number
merchnum_zip = df_clean.groupby('Merchnum')['Merch zip'].agg(pd.Series.mode).reset_index()

miss_num_zip = []
for idx in missing_zip:
    if df_clean.loc[idx, 'Merchnum'] in merchnum_zip['Merchnum'].values:
            numcode = df_clean.loc[idx, 'Merchnum']
            if type(merchnum_zip[merchnum_zip['Merchnum'] == numcode]['Merch zip'].values[0]) != np.ndarray:
                df_clean.loc[idx, 'Merch zip'] = merchnum_zip[merchnum_zip['Merchnum'] == numcode]['Merch zip'].values[0]
            else:
                miss_num_zip.append(idx)
    else:
        miss_num_zip.append(idx)
        
print(len(miss_num_zip))

#Fill in with unknown
for idx in miss_num_zip:
    df_clean.loc[idx, 'Merch zip'] = 'Unk'

3251
1104


In [13]:
sum(df_clean['Merch zip'].isnull())

0

## Create Variables

In [14]:
df_variable = df_clean.copy()#.head()
df_variable.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,1/1/10,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,1/1/10,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,1/1/10,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [15]:
#1. weekday variable
df_variable.Date = pd.to_datetime(df_variable.Date)
df_variable['weekday'] = df_variable.Date.dt.day_name()

### Encoding target variables (2)

In [16]:
#Include two target encoded variables: 
#likelihood of fraud for that day of the week and for that state. Don’t use the OOT data to calculate these 2 variables.

train_test = df_variable[df_variable.Date <= '2010-8-31']
day_prob = train_test.groupby('weekday')['Fraud'].mean().reset_index().rename(columns = {'Fraud':'day_prob'})
df_variable = df_variable.merge(day_prob, how='left', on='weekday')

In [17]:
state_prob = train_test.groupby('Merch state')['Fraud'].mean().reset_index().rename(columns = {'Fraud':'state_prob'})
df_variable = df_variable.merge(state_prob, how='left', on='Merch state')

In [18]:
prob = df_variable.columns[-2:]

### Amount +frequency variables (5 * 7 * 2 * 6)

In [19]:
df_variable['days'] = 0
day = 1
for i in range(len(df_variable)-1):
    if df_variable.loc[i, 'weekday'] == df_variable.loc[i+1, 'weekday']:
        df_variable.loc[i, 'days'] = day
    else:
        df_variable.loc[i, 'days'] = day
        day=day+1


In [20]:
df_variable.loc[96396, 'days'] = 365
df_variable['2_day'] = np.ceil(df_variable['days']/2)
df_variable['3_day'] = np.ceil(df_variable['days']/3)
df_variable['7_day'] = np.ceil(df_variable['days']/7)
df_variable['14_day'] = np.ceil(df_variable['days']/14)
df_variable['30_day'] = np.ceil(df_variable['days']/30)


In [21]:
#1 day
method = ['mean', 'max', 'median', 'sum', 'count']

card_1 = df_variable.groupby(['days', 'Cardnum'])['Amount'].agg(method).add_prefix('card_1_').reset_index()
df_variable = df_variable.merge(card_1, how = 'left', on=['days', 'Cardnum'])

Merchnum_1 = df_variable.groupby(['days', 'Merchnum'])['Amount'].agg(method).add_prefix('Merchnum_1_').reset_index()
df_variable = df_variable.merge(Merchnum_1, how = 'left', on=['days', 'Merchnum'])

MerchDesc_1 = df_variable.groupby(['days', 'Merch description'])['Amount'].agg(method).add_prefix('MerchDesc_1_').reset_index()
df_variable = df_variable.merge(MerchDesc_1, how = 'left', on=['days', 'Merch description'])

CMN_1 = df_variable.groupby(['days', 'Cardnum', 'Merchnum'])['Amount'].agg(method).add_prefix('CMN_1_').reset_index()
df_variable = df_variable.merge(CMN_1, how = 'left', on=['days', 'Cardnum', 'Merchnum'])

CMDesc_1 = df_variable.groupby(['days', 'Cardnum', 'Merch description'])['Amount'].agg(method).add_prefix('CMDesc_1_').reset_index()
df_variable = df_variable.merge(CMDesc_1, how = 'left', on=['days', 'Cardnum', 'Merch description'])

CZip_1 = df_variable.groupby(['days', 'Cardnum', 'Merch zip'])['Amount'].agg(method).add_prefix('CZip_1_').reset_index()
df_variable = df_variable.merge(CZip_1, how = 'left', on=['days', 'Cardnum', 'Merch zip'])

CState_1 = df_variable.groupby(['days', 'Cardnum', 'Merch state'])['Amount'].agg(method).add_prefix('CState_1_').reset_index()
df_variable = df_variable.merge(CState_1, how = 'left', on=['days', 'Cardnum', 'Merch state'])


In [22]:
amount1 = df_variable.columns[-35:]

In [23]:
for col in df_variable.columns[-35:]:
    name = 'Actual_' + col
    df_variable[name] = df_variable['Amount'] / df_variable[col]

In [24]:
#Amount Variables
#2 days
card_2 = df_variable.groupby(['2_day', 'Cardnum'])['Amount'].agg(method).add_prefix('card_2_').reset_index()
df_variable = df_variable.merge(card_2, how = 'left', on=['2_day', 'Cardnum'])

Merchnum_2 = df_variable.groupby(['2_day', 'Merchnum'])['Amount'].agg(method).add_prefix('Merchnum_2_').reset_index()
df_variable = df_variable.merge(Merchnum_2, how = 'left', on=['2_day', 'Merchnum'])

MerchDesc_2 = df_variable.groupby(['2_day', 'Merch description'])['Amount'].agg(method).add_prefix('MerchDesc_2_').reset_index()
df_variable = df_variable.merge(MerchDesc_2, how = 'left', on=['2_day', 'Merch description'])

CMN_2 = df_variable.groupby(['2_day', 'Cardnum', 'Merchnum'])['Amount'].agg(method).add_prefix('CMN_2_').reset_index()
df_variable = df_variable.merge(CMN_2, how = 'left', on=['2_day', 'Cardnum', 'Merchnum'])

CMDesc_2 = df_variable.groupby(['2_day', 'Cardnum', 'Merch description'])['Amount'].agg(method).add_prefix('CMDesc_2_').reset_index()
df_variable = df_variable.merge(CMDesc_2, how = 'left', on=['2_day', 'Cardnum', 'Merch description'])

CZip_2 = df_variable.groupby(['2_day', 'Cardnum', 'Merch zip'])['Amount'].agg(method).add_prefix('CZip_2_').reset_index()
df_variable = df_variable.merge(CZip_2, how = 'left', on=['2_day', 'Cardnum', 'Merch zip'])

CState_2 = df_variable.groupby(['2_day', 'Cardnum', 'Merch state'])['Amount'].agg(method).add_prefix('CState_2_').reset_index()
df_variable = df_variable.merge(CState_2, how = 'left', on=['2_day', 'Cardnum', 'Merch state'])

In [25]:
amount2 = df_variable.columns[-35:]

In [26]:
for col in df_variable.columns[-35:]:
    name = 'Actual_' + col
    df_variable[name] = df_variable['Amount'] / df_variable[col]

In [27]:
#Amount Variables
#3 days


card_3 = df_variable.groupby(['3_day', 'Cardnum'])['Amount'].agg(method).add_prefix('card_3_').reset_index()
df_variable = df_variable.merge(card_3, how = 'left', on=['3_day', 'Cardnum'])

Merchnum_3= df_variable.groupby(['3_day', 'Merchnum'])['Amount'].agg(method).add_prefix('Merchnum_3_').reset_index()
df_variable = df_variable.merge(Merchnum_3, how = 'left', on=['3_day', 'Merchnum'])

MerchDesc_3 = df_variable.groupby(['3_day', 'Merch description'])['Amount'].agg(method).add_prefix('MerchDesc_3_').reset_index()
df_variable = df_variable.merge(MerchDesc_3, how = 'left', on=['3_day', 'Merch description'])

CMN_3 = df_variable.groupby(['3_day', 'Cardnum', 'Merchnum'])['Amount'].agg(method).add_prefix('CMN_3_').reset_index()
df_variable = df_variable.merge(CMN_3, how = 'left', on=['3_day', 'Cardnum', 'Merchnum'])

CMDesc_3 = df_variable.groupby(['3_day', 'Cardnum', 'Merch description'])['Amount'].agg(method).add_prefix('CMDesc_3_').reset_index()
df_variable = df_variable.merge(CMDesc_3, how = 'left', on=['3_day', 'Cardnum', 'Merch description'])

CZip_3 = df_variable.groupby(['3_day', 'Cardnum', 'Merch zip'])['Amount'].agg(method).add_prefix('CZip_3_').reset_index()
df_variable = df_variable.merge(CZip_3, how = 'left', on=['3_day', 'Cardnum', 'Merch zip'])

CState_3 = df_variable.groupby(['3_day', 'Cardnum', 'Merch state'])['Amount'].agg(method).add_prefix('CState_3_').reset_index()
df_variable = df_variable.merge(CState_3, how = 'left', on=['3_day', 'Cardnum', 'Merch state'])

In [28]:
amount3 = df_variable.columns[-35:]

In [29]:
for col in df_variable.columns[-35:]:
    name = 'Actual_' + col
    df_variable[name] = df_variable['Amount'] / df_variable[col]

In [30]:
#Amount variables
#7 days

card_7 = df_variable.groupby(['7_day', 'Cardnum'])['Amount'].agg(method).add_prefix('card_7_').reset_index()
df_variable = df_variable.merge(card_7, how = 'left', on=['7_day', 'Cardnum'])

Merchnum_7= df_variable.groupby(['7_day', 'Merchnum'])['Amount'].agg(method).add_prefix('Merchnum_7_').reset_index()
df_variable = df_variable.merge(Merchnum_7, how = 'left', on=['7_day', 'Merchnum'])

MerchDesc_7 = df_variable.groupby(['7_day', 'Merch description'])['Amount'].agg(method).add_prefix('MerchDesc_7_').reset_index()
df_variable = df_variable.merge(MerchDesc_7, how = 'left', on=['7_day', 'Merch description'])

CMN_7 = df_variable.groupby(['7_day', 'Cardnum', 'Merchnum'])['Amount'].agg(method).add_prefix('CMN_7_').reset_index()
df_variable = df_variable.merge(CMN_7, how = 'left', on=['7_day', 'Cardnum', 'Merchnum'])

CMDesc_7 = df_variable.groupby(['7_day', 'Cardnum', 'Merch description'])['Amount'].agg(method).add_prefix('CMDesc_7_').reset_index()
df_variable = df_variable.merge(CMDesc_7, how = 'left', on=['7_day', 'Cardnum', 'Merch description'])

CZip_7 = df_variable.groupby(['7_day', 'Cardnum', 'Merch zip'])['Amount'].agg(method).add_prefix('CZip_7_').reset_index()
df_variable = df_variable.merge(CZip_7, how = 'left', on=['7_day', 'Cardnum', 'Merch zip'])

CState_7 = df_variable.groupby(['7_day', 'Cardnum', 'Merch state'])['Amount'].agg(method).add_prefix('CState_7_').reset_index()
df_variable = df_variable.merge(CState_7, how = 'left', on=['7_day', 'Cardnum', 'Merch state'])


In [31]:
amount7 = df_variable.columns[-35:]

In [32]:
for col in df_variable.columns[-35:]:
    name = 'Actual_' + col
    df_variable[name] = df_variable['Amount'] / df_variable[col]

In [33]:
#Amount variables
#14 days

card_14 = df_variable.groupby(['14_day', 'Cardnum'])['Amount'].agg(method).add_prefix('card_14_').reset_index()
df_variable = df_variable.merge(card_14, how = 'left', on=['14_day', 'Cardnum'])

Merchnum_14= df_variable.groupby(['14_day', 'Merchnum'])['Amount'].agg(method).add_prefix('Merchnum_14_').reset_index()
df_variable = df_variable.merge(Merchnum_14, how = 'left', on=['14_day', 'Merchnum'])

MerchDesc_14 = df_variable.groupby(['14_day', 'Merch description'])['Amount'].agg(method).add_prefix('MerchDesc_14_').reset_index()
df_variable = df_variable.merge(MerchDesc_14, how = 'left', on=['14_day', 'Merch description'])


CMN_14 = df_variable.groupby(['14_day', 'Cardnum', 'Merchnum'])['Amount'].agg(method).add_prefix('CMN_14_').reset_index()
df_variable = df_variable.merge(CMN_14, how = 'left', on=['14_day', 'Cardnum', 'Merchnum'])

CMDesc_14 = df_variable.groupby(['14_day', 'Cardnum', 'Merch description'])['Amount'].agg(method).add_prefix('CMDesc_14_').reset_index()
df_variable = df_variable.merge(CMDesc_14, how = 'left', on=['14_day', 'Cardnum', 'Merch description'])

CZip_14 = df_variable.groupby(['14_day', 'Cardnum', 'Merch zip'])['Amount'].agg(method).add_prefix('CZip_14_').reset_index()
df_variable = df_variable.merge(CZip_14, how = 'left', on=['14_day', 'Cardnum', 'Merch zip'])

CState_14 = df_variable.groupby(['14_day', 'Cardnum', 'Merch state'])['Amount'].agg(method).add_prefix('CState_14_').reset_index()
df_variable = df_variable.merge(CState_14, how = 'left', on=['14_day', 'Cardnum', 'Merch state'])


In [34]:
amount14 = df_variable.columns[-35:]

In [35]:
for col in df_variable.columns[-35:]:
    name = 'Actual_' + col
    df_variable[name] = df_variable['Amount'] / df_variable[col]

In [36]:
#Amount variables
#30 days

card_30 = df_variable.groupby(['30_day', 'Cardnum'])['Amount'].agg(method).add_prefix('card_30_').reset_index()
df_variable = df_variable.merge(card_30, how = 'left', on=['30_day', 'Cardnum'])

Merchnum_30= df_variable.groupby(['30_day', 'Merchnum'])['Amount'].agg(method).add_prefix('Merchnum_30_').reset_index()
df_variable = df_variable.merge(Merchnum_30, how = 'left', on=['30_day', 'Merchnum'])

MerchDesc_30 = df_variable.groupby(['30_day', 'Merch description'])['Amount'].agg(method).add_prefix('MerchDesc_30_').reset_index()
df_variable = df_variable.merge(MerchDesc_30, how = 'left', on=['30_day', 'Merch description'])

CMN_30 = df_variable.groupby(['30_day', 'Cardnum', 'Merchnum'])['Amount'].agg(method).add_prefix('CMN_30_').reset_index()
df_variable = df_variable.merge(CMN_30, how = 'left', on=['30_day', 'Cardnum', 'Merchnum'])

CMDesc_30 = df_variable.groupby(['30_day', 'Cardnum', 'Merch description'])['Amount'].agg(method).add_prefix('CMDesc_30_').reset_index()
df_variable = df_variable.merge(CMDesc_30, how = 'left', on=['30_day', 'Cardnum', 'Merch description'])

CZip_30 = df_variable.groupby(['30_day', 'Cardnum', 'Merch zip'])['Amount'].agg(method).add_prefix('CZip_30_').reset_index()
df_variable = df_variable.merge(CZip_30, how = 'left', on=['30_day', 'Cardnum', 'Merch zip'])

CState_30 = df_variable.groupby(['30_day', 'Cardnum', 'Merch state'])['Amount'].agg(method).add_prefix('CState_30_').reset_index()
df_variable = df_variable.merge(CState_30, how = 'left', on=['30_day', 'Cardnum', 'Merch state'])


In [37]:
amount30 = df_variable.columns[-35:]

In [38]:
for col in df_variable.columns[-35:]:
    name = 'Actual_' + col
    df_variable[name] = df_variable['Amount'] / df_variable[col]

In [39]:
Amount_freq = df_variable.columns[-420:]
Amount_freq

Index(['card_1_mean', 'card_1_max', 'card_1_median', 'card_1_sum',
       'card_1_count', 'Merchnum_1_mean', 'Merchnum_1_max',
       'Merchnum_1_median', 'Merchnum_1_sum', 'Merchnum_1_count',
       ...
       'Actual_CZip_30_mean', 'Actual_CZip_30_max', 'Actual_CZip_30_median',
       'Actual_CZip_30_sum', 'Actual_CZip_30_count', 'Actual_CState_30_mean',
       'Actual_CState_30_max', 'Actual_CState_30_median',
       'Actual_CState_30_sum', 'Actual_CState_30_count'],
      dtype='object', length=420)

In [40]:
exclude = []
for var in Amount_freq:
    turn = var.split('_')
    if turn[0] == 'Actual' and turn[-1] == 'count':
        exclude.append(var)

In [41]:
Amount_freq_ud = []
for var in Amount_freq:
    if var not in exclude:
        Amount_freq_ud.append(var)
len(Amount_freq_ud)

378

### Days-since variables (6)

In [42]:
df_variable['Merch zip'] = df_variable['Merch zip'].astype(str)
df_variable['Cardnum'] = df_variable['Cardnum'].astype(object)

In [43]:
method = ['max']

def day_since(card, df, rsuffix, field1, field2, day):
    name = 'days_since' + rsuffix
    if name not in df.columns:
        df[name] = 365
    turn = df[df['days'] == day]  

    if field2 == 0:
        turn = turn.merge(card, how = 'left', on = field1)

    else:
        #print(card.info())
        turn = turn.merge(card, how = 'left', on = [field1, field2])
        
    for j in range(len(turn)):
        if turn.loc[j,'rec_date'] in [pd.NaT]:
            turn.loc[j, 'rec_date']  = turn.loc[j, 'Date']
            
    turn.loc[:, name] = turn.loc[:, 'Date'] - turn.loc[:, 'rec_date']
    turn.loc[:, name] = turn.loc[:, name]/ np.timedelta64(1, 'D')
    turn[name] = turn[name].replace(0, 365)
    turn = turn.set_index('Recnum')
    return(turn[[name]])

for i in range(364): #days=1~days=365
    i+=1

    past_df = df_variable[df_variable['days'] <= i]
    
    Card_ = past_df.groupby('Cardnum')['Date'].agg(method).reset_index().rename(columns={'max':'rec_date'})
    test_c = day_since(Card_, df_variable, '_card', 'Cardnum', 0, i+1)
    
    Merchnum_= past_df.groupby(['Merchnum'])['Date'].agg(method).reset_index().rename(columns={'max':'rec_date'})
    test_m1 = day_since(Merchnum_, df_variable, '_Merchnum', 'Merchnum', 0, i+1)
    
    MerchDesc_= past_df.groupby(['Merch description'])['Date'].agg(method).reset_index().rename(columns={'max':'rec_date'})
    test_m2 = day_since(MerchDesc_, df_variable, '_MerchDesc', 'Merch description', 0, i+1)
        
    CMN_ = past_df.groupby(['Cardnum', 'Merchnum'])['Date'].agg(method).reset_index().rename(columns={'max':'rec_date'})
    test_cmn = day_since(CMN_, df_variable, '_CMN','Cardnum', 'Merchnum', i+1)

    CMDesc_ = past_df.groupby(['Cardnum', 'Merch description'])['Date'].agg(method).reset_index().rename(columns={'max':'rec_date'})
    test_cmd = day_since(CMDesc_, df_variable, '_CMDesc', 'Cardnum', 'Merch description', i+1)

    CZip_ = past_df.groupby(['Cardnum', 'Merch zip'])['Date'].agg(method).reset_index().rename(columns={'max':'rec_date'})
    test_zip = day_since(CZip_, df_variable, '_CZip', 'Cardnum', 'Merch zip', i+1)

    CState_ = past_df.groupby(['Cardnum', 'Merch state'])['Date'].agg(method).reset_index().rename(columns={'max':'rec_date'})
    test_sta = day_since(CState_, df_variable, '_CState', 'Cardnum', 'Merch state', i+1)

    for i in range(len(df_variable)):
        if df_variable.loc[i, 'Recnum'] in test_c.index:
            idx = df_variable.loc[i, 'Recnum']
            df_variable.loc[i, 'days_since_card'] = test_c.loc[idx, 'days_since_card']
            df_variable.loc[i, 'days_since_Merchnum'] = test_m1.loc[idx, 'days_since_Merchnum']
            df_variable.loc[i, 'days_since_MerchDesc'] = test_m2.loc[idx, 'days_since_MerchDesc']
            df_variable.loc[i, 'days_since_CMN'] = test_cmn.loc[idx, 'days_since_CMN']
            df_variable.loc[i, 'days_since_CMDesc'] = test_cmd.loc[idx, 'days_since_CMDesc']
            df_variable.loc[i, 'days_since_CZip'] = test_zip.loc[idx, 'days_since_CZip']
            df_variable.loc[i, 'days_since_CState'] = test_sta.loc[idx, 'days_since_CState']



In [44]:
days_since_var = df_variable.columns[-7:]
days_since_var

Index(['days_since_card', 'days_since_Merchnum', 'days_since_MerchDesc',
       'days_since_CMN', 'days_since_CMDesc', 'days_since_CZip',
       'days_since_CState'],
      dtype='object')

### Velocity change Variables (2 * 3 * 2 * 3 = 36 )

In [45]:
numerator = ['card_1_count', 'card_2_count', #count = number of transactions
             'Merchnum_1_count', 'Merchnum_2_count', 
             'MerchDesc_1_count', 'MerchDesc_2_count',
             'card_1_sum', 'card_2_sum',  #sum = total amount of transactions
             'Merchnum_1_sum', 'Merchnum_2_sum', 
             'MerchDesc_1_sum', 'MerchDesc_2_sum']

denominator = ['card_7_count', 'card_14_count','card_30_count', #count = number of transactions
             'Merchnum_7_count', 'Merchnum_14_count', 'Merchnum_30_count',
             'MerchDesc_7_count', 'MerchDesc_14_count', 'MerchDesc_30_count',
             'card_7_sum', 'card_14_sum', 'card_30_sum', #sum = total amount of transactions
             'Merchnum_7_sum', 'Merchnum_14_sum', 'Merchnum_30_sum',
             'MerchDesc_7_sum', 'MerchDesc_14_sum', 'MerchDesc_30_sum']


In [46]:
#df_variable[denominator]
import re
for den in denominator:
    m = re.search('(\d)', den)
    name = den+'_avg'
    df_variable[name] = df_variable[den]/ int(m.group(0))


In [47]:
denominator = [(den + '_avg') for den in denominator]

In [48]:
for num in numerator:
    for den in denominator:
        if (num.split('_')[0] == den.split('_')[0]) & (num.split('_')[2] == den.split('_')[2]):
            var = num.split('_')[0] + '_'+ num.split('_')[2] + '_' + num.split('_')[1] +'_'+ den.split('_')[1]
            df_variable[var] = df_variable[num] / df_variable[den]

In [49]:
velocity_var = df_variable.columns[-36:]

## Put all together

In [50]:
a = prob.to_list() # 2 target encoded variables
b = Amount_freq_ud # 378 amount and frequency variables
c = days_since_var.to_list() # 7 days since variables
d = velocity_var.to_list() #36 velocity variables
all_var = a+b+c+d
len(all_var)

423

In [51]:
nyvar = df_variable[all_var]
submit = pd.DataFrame(round(nyvar.max(), 3), columns = ['max'])
submit['min'] = nyvar.min()
submit['mean'] = round(nyvar.mean(), 3)
submit['stdev'] = round(nyvar.std(), 3)
submit.reset_index().rename(columns={'index':'variable name'})

,variable name,max,min,mean,stdev
0,day_prob,0.040,0.006652,0.011,0.007
1,state_prob,0.057,0.000000,0.010,0.009
2,card_1_mean,28392.840,0.140000,395.858,703.748
3,card_1_max,47900.000,0.140000,586.792,1195.496
4,card_1_median,28392.840,0.140000,370.775,688.370
...,...,...,...,...,...
418,MerchDesc_sum_1_14,1.000,0.000084,0.557,0.383
419,MerchDesc_sum_1_30,3.000,0.000089,1.334,1.161
420,MerchDesc_sum_2_7,527.516,0.001467,5.254,3.154
421,MerchDesc_sum_2_14,1.000,0.000084,0.603,0.365


In [52]:
#submit.to_csv('card_tans_var_statistics.csv')

In [53]:
#nyvar.to_csv('card_variable_list.csv')

In [58]:
df_filter = df_variable[df_variable.days >14]

In [62]:
df_filter = df_filter[df_filter.Date <= '2010-08-31']

In [67]:
filter_var = train = all_var + ['Fraud']
df_filter[filter_var].to_csv('filter_variable_list.csv')

In [68]:
df_variable.to_csv('all_df_var.csv')